In [ ]:
if 'filenames' not in locals() and 'filenames' not in globals():
    SETUP_PATH = "/home/xjaros2/Documents/git/csidh-setup/"
    filenames = ["./data/parameter-search/husky/dummy/husky-clock-param-search-random-(+5)(-3)(+1)-initial.json"]
    BINS = 50
    OUTPUT_FILE = "parameter_search_initial.pdf"
    %cd $SETUP_PATH/notebooks

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from csidhtools import Unit
from scipy.signal import find_peaks
from matplotlib.patches import Patch

TITLE_FONTSIZE = 16
YLABEL_FONTSIZE = 13
XLABEL_FONTSIZE = 13

plt.rcParams.update({
    "text.usetex": True,
    "ytick.color" : "black",
    "xtick.color" : "black",
    "axes.labelcolor" : "black",
    "axes.edgecolor" : "black",
    "font.family" : "serif",
    "font.serif" : ["Computer Modern Serif"]
})

In [ ]:
from csidhtools.search.io import read_caches_into_dataframe
df = read_caches_into_dataframe(filenames)

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
import matplotlib.gridspec as gridspec

# --- typemap ---
typemap = {
    "JUSTRIGHT": "Faulty",
    "NORMAL": "Normal",
    "RESET": "Reset",
    "CHANGING": "Changing"
}

# --- hue order & colors ---
hue_order = ["NORMAL", "RESET", "CHANGING", "JUSTRIGHT"]
palette   = ["#1E88E5", "#D81B60", "#FFC187", "#04AB8F"]

# Drop CHANGING if missing
if df[df.type == "CHANGING"].empty:
    hue_order.remove("CHANGING")
    palette.remove("#FFC187")

# User option
show_kdes = False   # <--- TOGGLE KDE MARGINALS HERE


# ============================================================
#              FIGURE LAYOUT (with or without KDEs)
# ============================================================

if show_kdes:
    fig = plt.figure(figsize=(8, 8))
    gs = gridspec.GridSpec(
        2, 2,
        width_ratios=[4, 1],
        height_ratios=[1, 4],
        wspace=0.0,
        hspace=0.0
    )

    ax_main  = fig.add_subplot(gs[1, 0])
    ax_top   = fig.add_subplot(gs[0, 0], sharex=ax_main)
    ax_right = fig.add_subplot(gs[1, 1], sharey=ax_main)

    # Clean marginals
    for ax in [ax_top, ax_right]:
        for s in ax.spines.values():
            s.set_visible(False)
        ax.tick_params(
            left=False, right=False, bottom=False, top=False,
            labelleft=False, labelbottom=False
        )

    plt.setp(ax_top.get_xticklabels(), visible=False)
    plt.setp(ax_right.get_yticklabels(), visible=False)

else:
    # Single plot figure
    fig, ax_main = plt.subplots(figsize=(7, 7))
    ax_top = None
    ax_right = None


# ============================================================
#                BIN EDGES (NO SEABORN PADDING)
# ============================================================

x = df["width"].to_numpy()
y = df["offset"].to_numpy()

x_edges = np.linspace(x.min(), x.max(), BINS + 1)
y_edges = np.linspace(y.min(), y.max(), BINS + 1)


# ============================================================
#               MAIN 2D HIST + OPTIONAL KDEs
# ============================================================

for i, (t, p) in enumerate(zip(hue_order, palette)):
    subset = df[df["type"] == t]

    # --- Main 2D seaborn histplot using explicit edges ---
    sns.histplot(
        subset,
        x="width",
        y="offset",
        bins=[x_edges, y_edges],
        ax=ax_main,
        alpha=1,
        cbar=False,
        element="poly",
    )

    # Force solid color
    quad = ax_main.collections[-1]
    quad.set_cmap(mcolors.ListedColormap([p]))
    quad.set_clim(0, 1)
    quad.set_zorder(i)

    # --- OPTIONALLY ADD KDEs ---
    if show_kdes:
        sns.kdeplot(
            data=subset,
            x="width",
            ax=ax_top,
            color=p,
            linewidth=2,
            fill=False
        )
        sns.kdeplot(
            data=subset,
            y="offset",
            ax=ax_right,
            color=p,
            linewidth=2,
            fill=False
        )


# ============================================================
#                         LABELS + LEGEND
# ============================================================

ax_main.set_xlabel("Width", fontsize=14)
ax_main.set_ylabel("Offset", fontsize=14)
ax_main.set_title("Width vs Offset", fontsize=18, y=1.04)

legend_elements = [
    Patch(facecolor=p, label=typemap[t])
    for (t, p) in zip(hue_order, palette)
]

fig.legend(
    handles=legend_elements,
    title="Type",
    loc="upper right",
    bbox_to_anchor=(1.0, 1.0),
    frameon=True,
    borderpad=0.7
)


# ============================================================
#                     ZERO-MARGIN CLEANUP
# ============================================================

ax_main.set_xlim(x.min(), x.max())
ax_main.set_ylim(y.min(), y.max())

ax_main.set_xmargin(0)
ax_main.set_ymargin(0)
ax_main.margins(0, 0)

ax_main.relim()
ax_main.autoscale_view()

plt.tight_layout(pad=0)
plt.savefig(OUTPUT_FILE, bbox_inches="tight", pad_inches=0)
